Task 1
1. ownself explore the job and user dataset
2. Merge all the job skills related column and user want to work with column
3. Learn BERT and BI LSTM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/bt4222_notebook/

[Errno 2] No such file or directory: 'drive/MyDrive/bt4222_notebook/'
/content/drive/MyDrive/bt4222_notebook


In [ ]:
jobs_df = pd.read_csv('embedded_jobs.csv')
print(jobs_df['company'].size)
# print(jobs_df.head(3))
print(jobs_df['uniq_id'].value_counts())
print(jobs_df['embeddings'][666])

22000
418ff92580b270ef4e7c14f0ddfc36b4    1
49574eb8f13fa8e4a80fd2f857aba943    1
16f3f5359808e4332e7f54d8e93fbeb7    1
5d849a66db4287b7da5d387da45f1210    1
968fc7d79f3ec9d1b436661b2a52ae5b    1
                                   ..
e03ce5b64be2b630b418183a5816215a    1
8b8819f7d53312262ae5837f698e851c    1
87c4d73f9bc1748e22861c3f6e51d2b6    1
7945bd1d6bf48c167d84e4b7a747d674    1
9a4e8c27f74af4c0d2f6efbd420a8a91    1
Name: uniq_id, Length: 22000, dtype: int64
[ 0.9159354  -1.0536022   0.5528108   2.9904435  -0.41632506 -0.42947528
  0.27432713 -1.0200375  -2.2026553  -1.838965    0.3191411  -2.3289955
  0.18697186 -2.4641862   0.9152056  -1.5789089  -0.01105886 -2.6982281
 -0.5237386   0.9163731   0.7335649   0.18095453  1.4053663  -0.4206053
  0.5669052   0.1089344  -0.02278094  2.6474874   0.8326472   0.19625266
 -0.6969066   1.135774    0.9019868   1.0584711   0.5668459   0.9802674
 -0.4853691   0.9479676   0.27626038  1.4949781   0.36973393 -0.9636729
  0.35604683  0.06174603  2

In [ ]:
users_df = pd.read_csv('embedded_users.csv')
users_df['Age'].size

50000

In [ ]:
print(users_df['embeddings'][66])

[-6.5038390e-03  7.8010865e-02  5.3762770e-01  2.1152560e-01
  6.6291153e-02 -1.9286983e-01  2.2406350e-03 -9.1684125e-02
  3.9619006e-02 -8.6804688e-02  1.9246997e-01 -3.4549788e-01
  3.8902801e-02 -1.2203430e-01  1.7773153e-01 -2.8679571e-01
  3.9135283e-01  1.5702112e-01 -1.6588344e-01 -4.8099712e-02
 -4.2905388e-04  2.4915414e-01  1.8827063e-01 -2.5911611e-01
  1.0062188e-01  2.2036646e-01  2.3152601e-02 -3.7530601e-02
 -5.6202862e-02 -1.1022873e-01 -1.8201165e-01 -4.4715341e-02
  6.9377422e-02  2.1037129e-01 -2.1131599e-01  1.5650806e-01
 -1.3936815e-01  1.8102627e-01 -3.3645940e-01  3.6616457e-01
  9.1167286e-02  2.4129849e-02  2.1012723e-01  7.5545525e-03
  4.3806064e-01 -1.3933705e-01 -5.5536353e-03 -1.3494815e-01
  3.2885319e-01  2.6464555e-01]


In [ ]:
# n_models = 10
# def create_bootstrap_samples(df, n_samples):
#     bootstrap_samples = [df.sample(n=n_samples, replace=True).reset_index(drop=True) for _ in range(n_models)]
#     return bootstrap_samples

sample_1_user = users_df.sample(n=2000, replace=True, random_state=0).reset_index(drop=True)
sample_1_job = jobs_df.sample(n=2000, replace=True, random_state=0).reset_index(drop=True)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

# def convert_str_to_array(embed_str):
#     list_representation = literal_eval(embed_str)
#     return np.array(list_representation)
def convert_str_to_array(embed_str):
    embed_str = embed_str[1:-1]
    str_nums = embed_str.strip().split()
    return np.array([float(num) for num in str_nums if num], dtype=float)

users_df['embeddings'] = users_df['embeddings'].apply(convert_str_to_array)
jobs_df['embeddings'] = jobs_df['embeddings'].apply(convert_str_to_array)

user_embeddings = np.stack(users_df['embeddings'].values)
job_embeddings = np.stack(jobs_df['embeddings'].values)
similarity_matrix = cosine_similarity(user_embeddings, job_embeddings)

In [ ]:
for label in ['top1_id', 'top1_score', 'top2_id', 'top2_score', 'bottom1_id', 'bottom1_score', 'bottom2_id', 'bottom2_score']:
    users_df[label] = None

# Iterate through each user to find top 2 and bottom 2 jobs based on similarity
for i, similarities in enumerate(similarity_matrix):
    # Get indices of jobs sorted by similarity for this user
    sorted_indices = np.argsort(similarities)

    # Top 2 - highest similarity scores
    top_indices = sorted_indices[-2:][::-1]  # Reverse to have top1 before top2
    users_df.at[i, 'top1_id'], users_df.at[i, 'top2_id'] = jobs_df.iloc[top_indices]['uniq_id'].values
    users_df.at[i, 'top1_score'], users_df.at[i, 'top2_score'] = similarities[top_indices]

    # Bottom 2 - lowest similarity scores
    bottom_indices = sorted_indices[:2]  # Already in ascending order
    users_df.at[i, 'bottom1_id'], users_df.at[i, 'bottom2_id'] = jobs_df.iloc[bottom_indices]['uniq_id'].values
    users_df.at[i, 'bottom1_score'], users_df.at[i, 'bottom2_score'] = similarities[bottom_indices]


In [ ]:
print(users_df.head(10))

   ResponseId     Q120                      MainBranch                 Age  \
0       22607  I agree  I am a developer by profession     35-44 years old   
1        5640  I agree  I am a developer by profession     25-34 years old   
2        1154  I agree  I am a developer by profession     55-64 years old   
3       76622  I agree           I am learning to code  Under 18 years old   
4        8538  I agree  I am a developer by profession     35-44 years old   
5        4560  I agree  I am a developer by profession     35-44 years old   
6       33635  I agree  I am a developer by profession     18-24 years old   
7       58490  I agree  I am a developer by profession     25-34 years old   
8       16570  I agree  I am a developer by profession     35-44 years old   
9       86356  I agree  I am a developer by profession     25-34 years old   

                                          Employment  \
0                                Employed, full-time   
1                            

In [ ]:
value = jobs_df.loc[jobs_df['uniq_id'] == 'ce23eb00a9b99f49d8ffd7b098983abc', 'Job_Features_Merged'].iloc[0]
print(value)
print(users_df['Tech_Features_Merged'][5])

Java Developer java java
C++;Go;Python;SQL Cassandra;Clickhouse;Dynamodb;MongoDB;PostgreSQL;Redis Apache Kafka;RabbitMQ CMake;Docker;Homebrew;Kubernetes;Make;Pip CLion;Goland;PyCharm;Vim;Visual Studio Code MacOS Confluence;Jira Discord;Zoom
